In [3]:
# This Python file uses the following encoding: utf-8
%matplotlib inline
import numpy as np
import pandas as pd
from English_to_IPA import conversion
import re
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from num2words import num2words

In [24]:
categories = ['Oral Cavity',
              'Mouth Openness',
              'Obstruent',
              'Tongue Positioning',
              'Resonance',
              'Vowel','Round','Tense']
eSPEDict = dict()
with open('eSPEPhonologicalTableV2') as openFile:
    for line in openFile.readlines():
        line = line.strip().split('\t')
        eSPEDict[line[0].decode('string_escape')] = np.array([eval(x) for x in line[1:]])

TypeError: 'encoding' is an invalid keyword argument for this function

In [35]:
text = line[0]
text.decode('Latin-1')

u'\xc9\x99'

In [37]:
def split_text(text):
    text = re.sub('\n+', '\n', text)
    splitted_text = re.sub(' +', ' ', clean_text(text)).split(' ')
    return [ word.lower() for word in splitted_text if word != '']

def clean_text(text):
    punct_str = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~«»“…‘”'
    for p in punct_str:
        text = text.replace(p,' ')
    return text  
    
def is_number(word):
    try:
        int(word)
        return True
    except ValueError:
        pass
    return False

def change_number_to_string(number):
    return num2words(int(number))

def is_empty(sequence):
    if len(sequence) == 0:
        return True
    return False

In [38]:
valid_dev = pd.read_csv('data/common_voice/cv-valid-dev.csv')
valid_dev.head()

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-dev/sample-000000.mp3,be careful with your prognostications said the...,1,0,NaN,NaN,NaN,NaN
1,cv-valid-dev/sample-000001.mp3,then why should they be surprised when they se...,2,0,NaN,NaN,NaN,NaN
2,cv-valid-dev/sample-000002.mp3,a young arab also loaded down with baggage ent...,2,0,NaN,NaN,NaN,NaN
3,cv-valid-dev/sample-000003.mp3,i thought that everything i owned would be des...,3,0,NaN,NaN,NaN,NaN
4,cv-valid-dev/sample-000004.mp3,he moved about invisible but everyone could he...,1,0,fourties,female,england,NaN


In [39]:
valid_train = pd.read_csv('data/common_voice/cv-valid-train.csv')
valid_train.head()

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-train/sample-000000.mp3,learn to recognize omens and follow them the o...,1,0,NaN,NaN,NaN,NaN
1,cv-valid-train/sample-000001.mp3,everything in the universe evolved he said,1,0,NaN,NaN,NaN,NaN
2,cv-valid-train/sample-000002.mp3,you came so that you could learn about your dr...,1,0,NaN,NaN,NaN,NaN
3,cv-valid-train/sample-000003.mp3,so now i fear nothing because it was those ome...,1,0,NaN,NaN,NaN,NaN
4,cv-valid-train/sample-000004.mp3,if you start your emails with greetings let me...,3,2,NaN,NaN,NaN,NaN


In [40]:
def get_score(text):
    word_count = 0
    score = np.zeros((8,), dtype=np.int)
    words = split_text(text)
    for word in words:
        print(word)
        if word == '':
            continue
        if is_number(word):
            #print("Found number: "+ word)
            word = change_number_to_string(word)
        cmu, ipa = conversion.convert(word)
        punct = '[ˌˈ ]'
        for p in punct:
            ipa = ipa.replace(p , '')
        if '*' in ipa:        
            #print('No IPA Found for: '+ word)
            continue
        word_count += 1
        i = 0
        print(ipa)
        while i < len(ipa):
            if i == len(ipa)-1:
                sym = ipa[i]
                score += eSPEDict[sym]
                i += 1
            else:
                try:
                    sym = ipa[i] + ipa[i+1]
                    score += eSPEDict[sym]
                    i += 2
                except KeyError:
                    sym = ipa[i]
                    score += eSPEDict[sym]
                    i += 1
    score = np.append(score, word_count)
    return score

In [45]:
valid_dev['energy_score'] = valid_dev['text'].apply(get_score)

be
bi
careful
kɛrfəl
with
wɪð
your
jʊr
prognostications
prɑgnɑstəkeɪʃənz


KeyError: 'e'

In [42]:
get_score('prognostications')

prognostications
prɑgnɑstəkeɪʃənz


KeyError: 'e'

NameError: name 'eSPEDict' is not defined